In [1]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_excel("British Airways Summer Schedule Dataset - Forage Data Science Task 1.xlsx")
print(df.head())
print(df.columns)
print(df.shape)

  FLIGHT_DATE FLIGHT_TIME TIME_OF_DAY AIRLINE_CD FLIGHT_NO  \
0  2025-09-02    14:19:00   Afternoon         BA    BA5211   
1  2025-06-10    06:42:00     Morning         BA    BA7282   
2  2025-10-27    15:33:00   Afternoon         BA    BA1896   
3  2025-06-15    18:29:00     Evening         BA    BA5497   
4  2025-08-25    20:35:00     Evening         BA    BA1493   

  DEPARTURE_STATION_CD ARRIVAL_STATION_CD ARRIVAL_COUNTRY ARRIVAL_REGION  \
0                  LHR                LAX             USA  North America   
1                  LHR                LAX             USA  North America   
2                  LHR                FRA         Germany         Europe   
3                  LHR                IST          Turkey         Europe   
4                  LHR                FRA         Germany         Europe   

    HAUL AIRCRAFT_TYPE  FIRST_CLASS_SEATS  BUSINESS_CLASS_SEATS  \
0   LONG          B777                  8                    49   
1   LONG          B777              

In [39]:
#Categorizing the data into one usable grouping
df["Category"]  = df["HAUL"] + " - " + df["ARRIVAL_REGION"]+" - " + df["TIME_OF_DAY"]

In [43]:
#Total numbers of passengers
df["Total_Passengers"]=(df["FIRST_CLASS_SEATS"] + df["BUSINESS_CLASS_SEATS"]+ df["ECONOMY_SEATS"])

#making assumptions & applying them to each of the row
assumptions = {"LONG - North America - Afternoon" : {"Tier1":0.03,"Tier2":0.1,"Tier3":0.28},"SHORT - Europe - Morning" : {"Tier1":0.01,"Tier2":0.05,"Tier3":0.12},"DOMESTIC - UK - Evening":{"Tier1":0.0005,"Tier2":0.02,"Tier3":0.06},}

               #Applying
def apply_lounge_model(row):
    category = row["Category"]
    Total = row["Total_Passengers"]
    if category in assumptions:
        tier1 = Total * assumptions[category]["Tier1"]
        tier2 = Total * assumptions[category]["Tier2"]
        tier3 = Total * assumptions[category]["Tier3"]
    else:
        tier1 = tier2 = tier3 = 0 
    return pd.Series([tier1,tier2,tier3])

df[["Est_tier1","Est_tier2","Est_tier3"]] = df.apply(apply_lounge_model, axis=1)

In [47]:
df[["Category","ARRIVAL_COUNTRY", "Total_Passengers", "Est_tier1", "Est_tier2", "Est_tier3"]].head()

,Category,ARRIVAL_COUNTRY,Total_Passengers,Est_tier1,Est_tier2,Est_tier3
0,LONG - North America - Afternoon,USA,235,7.05,23.5,65.8
1,LONG - North America - Morning,USA,235,0.00,0.0,0.0
2,SHORT - Europe - Afternoon,Germany,180,0.00,0.0,0.0
3,SHORT - Europe - Evening,Turkey,180,0.00,0.0,0.0
4,SHORT - Europe - Evening,Germany,180,0.00,0.0,0.0


In [55]:
# First, get the most common ARRIVAL_COUNTRY for each Category
country_mode = (
    df.groupby("Category")["ARRIVAL_COUNTRY"]
    .agg(lambda x: x.mode()[0])  # Pick the most frequent
    .reset_index()
)

# Now do your lounge tier summary
summary = (
    df.groupby("Category")[["Est_tier1", "Est_tier2", "Est_tier3"]]
    .sum()
    .round(1)
    .reset_index()
)

# Merge both to get ARRIVAL_COUNTRY into your summary
final_summary = pd.merge(summary, country_mode, on="Category")

# Save the final table to Excel
final_summary.to_excel("lounge_model_summary_ans.xlsx", index=False)